<a href="https://colab.research.google.com/github/amanbs/ECG_classification/blob/master/QRS_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import wfdb
import numpy as np
import scipy.signal as signal

# Load the ECG signal from a file
record = wfdb.rdrecord('filename')
signal = record.p_signal.flatten()

# Preprocess the signal
fs = record.fs
fc_low = 0.5 # Lower cutoff frequency for the band-pass filter
fc_high = 40 # Higher cutoff frequency for the band-pass filter
b, a = signal.butter(2, [fc_low/(fs/2), fc_high/(fs/2)], 'band')
filtered_signal = signal.filtfilt(b, a, signal)

# Detect the QRS complex
qrs_detector = None # Use a QRS detection algorithm like Pan-Tompkins or Wavelet Transform

# Measure the QT interval
qt_interval = None # Use an algorithm like the tangent method or the threshold-based method to detect the end of the T wave

# Calculate the corrected QT interval (QTc)
rr_interval = None # Calculate the RR interval between successive QRS complexes
heart_rate = None # Calculate the heart rate from the RR interval
qtc = qt_interval / np.sqrt(rr_interval) # Apply the Bazett's formula to correct the QT interval for heart rate variability

print("Corrected QT interval (QTc): ", qtc)


The function takes as input the ECG signal and the sampling frequency, and returns the indices of the QRS complex in the signal. You can then use these indices to measure the QT interval and calculate the corrected QT interval

In [ ]:
import numpy as np
import scipy.signal as signal

def pan_tompkins_detector(ecg_signal, fs):
    # Define filter parameters
    fc_low = 5 # Lower cutoff frequency for the high-pass filter
    fc_high = 15 # Higher cutoff frequency for the low-pass filter
    b_high, a_high = signal.butter(1, fc_low/(fs/2), 'highpass')
    b_low, a_low = signal.butter(1, fc_high/(fs/2), 'lowpass')

    # Apply the high-pass filter to remove baseline wander and low-frequency noise
    filtered_signal = signal.filtfilt(b_high, a_high, ecg_signal)

    # Square the signal to enhance QRS complexes
    squared_signal = filtered_signal ** 2

    # Apply the low-pass filter to remove high-frequency noise
    filtered_squared_signal = signal.filtfilt(b_low, a_low, squared_signal)

    # Find the moving average of the filtered signal to detect QRS complexes
    window_size = int(0.15 * fs)
    mov_avg = np.convolve(filtered_squared_signal, np.ones(window_size)/window_size, mode='valid')

    # Find the peaks of the moving average to detect QRS complexes
    peaks, _ = signal.find_peaks(mov_avg, height=0.4*np.max(mov_avg))

    # Remove false positive detections
    diff_peaks = np.diff(peaks)
    rr_avg = np.mean(diff_peaks)
    min_rr = int(0.6 * rr_avg)
    max_rr = int(1.5 * rr_avg)
    valid_peaks = [peaks[0]]
    for i in range(1, len(peaks)):
        if peaks[i] - valid_peaks[-1] > max_rr:
            valid_peaks.append(peaks[i])
        elif peaks[i] - valid_peaks[-1] >= min_rr:
            if mov_avg[peaks[i]] > mov_avg[valid_peaks[-1]]:
                valid_peaks[-1] = peaks[i]

    return valid_peaks


The function takes as input the ECG signal, the indices of the QRS complex in the signal, and the sampling frequency, and returns the QT interval in samples

In [ ]:
def tangent_method(ecg_signal, qrs_indices, fs):
    qt_interval = None
    
    # Define the search range for the end of the T wave
    search_range = int(0.4 * fs)

    for i in range(len(qrs_indices)-1):
        qrs_start = qrs_indices[i]
        qrs_end = qrs_indices[i+1]

        # Find the maximum point in the T wave
        t_peak = np.argmax(ecg_signal[qrs_end:qrs_end+search_range]) + qrs_end

        # Find the intersection point of the tangent line at the T peak and the isoelectric line
        tangent_slope = (ecg_signal[t_peak] - ecg_signal[t_peak-10]) / 10
        isoelectric_level = np.mean(ecg_signal[qrs_start-10:qrs_start])
        tangent_intercept = ecg_signal[t_peak] - tangent_slope * t_peak
        intersection_point = int((isoelectric_level - tangent_intercept) / tangent_slope)

        # Calculate the QT interval
        qt_interval_i = intersection_point - qrs_start

        # Update the QT interval if it's longer than the previous one
        if qt_interval is None or qt_interval_i > qt_interval:
            qt_interval = qt_interval_i

    return qt_interval
